In [42]:
import torch
import random
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
from sklearn.model_selection import train_test_split
import config
import torch 
import pandas as pd
from PIL import Image 
from os import listdir
from os.path import isfile, join
import torchvision.transforms as transforms 
import pandas as pd
import math

In [43]:
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.random.seed = random_seed

In [44]:
train_csv = pd.read_csv("../train_labels.csv")

path = '../images/images/train/'

train_features = []
train_targets = []

transform = transforms.Compose([ 
    transforms.Resize((config.size, config.size)),
    transforms.PILToTensor(),
]) 

if config.grayscale:
    transform = transforms.Compose([transforms.Grayscale(), transform])

for file in listdir(path):
    filename = join(path, file)
    if isfile(filename):
        image = Image.open(filename)
        width, height = image.size
        img_tensor = transform(image).type(torch.FloatTensor) 
        train_features.append(img_tensor)
        train_targets.append(train_csv.loc[train_csv['file_name'] == file].drop('file_name', axis = 1).values[0])
        for i in range(0, len(train_targets[-1]),2):
            train_targets[-1][i] = train_targets[-1][i]*(size1/width)
            train_targets[-1][i+1] = train_targets[-1][i+1]*(size2/height)

In [45]:
# from torch.utils.data import DataLoader
# image_data_loader = DataLoader(
#     train_features,
#     # batch size is whole dataset
#     batch_size=len(train_features),
#     shuffle=False,
#     num_workers=0)
# def mean_std(loader):
#   imgs = next(iter(loader))
#   # shape of images = [b,c,w,h]
#   mean, std = imgs.mean([0,2,3]), imgs.std([0,2,3])
#   return mean, std
# mean, std = mean_std(image_data_loader)
# print("mean and std: \n", mean, std)

In [46]:
X_train, X_test, Y_train, Y_test = train_test_split(train_features, train_targets, test_size= config.train_split_size,  random_state= random_seed) 

train_dataset = torch.utils.data.TensorDataset(torch.stack(X_train).type('torch.FloatTensor'), torch.FloatTensor(Y_train))
test_dataset = torch.utils.data.TensorDataset(torch.stack(X_test).type('torch.FloatTensor'), torch.FloatTensor(Y_test))

In [47]:
s = config.get_s()

In [48]:
torch.save(train_dataset, f'../datasets/train{s}.pt')
torch.save(test_dataset, f'../datasets/test{s}.pt')

In [49]:
path = '../images/images/test/'

predict_features = []

sizez = []
for file in listdir(path):
    filename = join(path, file)
    if isfile(filename):
        image = Image.open(filename)
        width, height = image.size
        sizez.append([width, height,file])
        img_tensor = transform(image) 
        predict_features.append(img_tensor)

In [50]:
predict_dataset = torch.utils.data.TensorDataset(torch.stack(predict_features).type('torch.FloatTensor'))
sizez = pd.DataFrame(sizez)

In [51]:
torch.save(predict_dataset, f'../datasets/predict{s}.pt')
sizez.to_csv('../datasets/predictSizez&names.csv')